In [1]:
# 1. 安装 PyTorch（如已安装，可跳过）
!pip install torch torchvision torchaudio --quiet

# 2. 安装 PyTorch Forecasting 和依赖
!pip install pytorch-forecasting pytorch-lightning --quiet

!pip install ipywidgets==7.7.1
!jupyter nbextension enable --py widgetsnbextension


  Using cached ipywidgets-7.7.1-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached widgetsnbextension-3.6.10-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached uri_template-1.3.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached webcolors-24.11.1-py3-none-any.whl.metadata (2.2 kB)
Using cached ipywidgets-7.7.1-py2.py3-none-any.whl (123 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.7 kB/s eta 0:00:0000:0100:02m
Using cached webcolors-24.11.1-py3-none-any.whl (14 kB)
Using cached fqdn-1.5.1-py3-none-any.whl (9.1 kB)
Using cached isoduration-20.11.0-py3-none-any.whl (11 kB)
Using cached uri_template-1.3.0-py3-none-any.whl (11 kB)
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.5.2
    Uninstalling widgetsnbextension-3.5.2:
      Successfully uninstalled widgetsnbextension-3.5.2
  Attempting

In [3]:
# ----------------------------
# 📦 1. 导入包
# ----------------------------
import numpy as np
import pandas as pd
import torch
from pytorch_forecasting import TimeSeriesDataSet, DeepAR
from pytorch_forecasting.data import GroupNormalizer
from lightning.pytorch import Trainer
import warnings
warnings.filterwarnings("ignore")

# 设置 float32 默认
torch.set_default_dtype(torch.float32)

# ----------------------------
# 🏗️ 2. 模拟数据集（你也可以换成真实数据）
# ----------------------------
np.random.seed(42)

n_advisors = 10
n_funds = 5
days = pd.date_range("2024-01-01", "2024-12-31", freq="D")
provinces = ['ON', 'BC', 'QC', 'AB']
data = []

for advisor_id in range(1, n_advisors + 1):
    province = np.random.choice(provinces)
    for fund_id in range(1, n_funds + 1):
        fund_name = f"Fund_{fund_id}"
        base_purchase = np.random.uniform(500, 1500)
        fund_trend = np.random.normal(0.1, 0.02)
        perf = 1.0

        for date in days:
            weekday = date.weekday()
            seasonal = 1.0 + 0.2 * np.sin(2 * np.pi * weekday / 7)
            purchase_amount = base_purchase * seasonal * perf + np.random.normal(0, 50)
            redem_amount = purchase_amount * np.random.uniform(0.05, 0.3)
            perf *= 1 + fund_trend + np.random.normal(0, 0.005)

            data.append({
                "financial_advisor_id": f"A{advisor_id:03d}",
                "fund_name": fund_name,
                "province": province,
                "trade_date": date,
                "purchase_amount": max(purchase_amount, 0),
                "redem_amount": max(redem_amount, 0),
                "fund_performance": perf
            })

df = pd.DataFrame(data)

# ----------------------------
# 🧼 3. 数据预处理
# ----------------------------
# 日期转换为 time_idx
df["time_idx"] = (df["trade_date"] - df["trade_date"].min()).dt.days

# 类别列转 category 类型
df["province"] = df["province"].astype("category")

# 所有 float 转为 float32，防止 MPS 报错
float_cols = df.select_dtypes(include=["float", "float64"]).columns
df[float_cols] = df[float_cols].astype("float32")

# ----------------------------
# ✂️ 4. 构建 TimeSeriesDataSet
# ----------------------------
max_encoder_length = 300
max_prediction_length = 31
training_cutoff = df["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df[df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="purchase_amount",
    group_ids=["financial_advisor_id", "fund_name"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["province"],
    time_varying_known_reals=["time_idx", "fund_performance"],
    time_varying_unknown_reals=["purchase_amount"],
    target_normalizer=GroupNormalizer(groups=["financial_advisor_id", "fund_name"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# 验证集（使用 .from_dataset 快速构建）
validation = TimeSeriesDataSet.from_dataset(
    training, df, predict=True, stop_randomization=True
)

# ----------------------------
# 📦 5. Dataloader
# ----------------------------
batch_size = 64
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

# ----------------------------
# 🧠 6. DeepAR 模型初始化
# ----------------------------
deepar = DeepAR.from_dataset(
    training,
    learning_rate=1e-3,
    hidden_size=32,
    dropout=0.1,
    log_interval=10,
    log_val_interval=1
)

# ----------------------------
# 🚀 7. Trainer 训练器（MPS 兼容）
# ----------------------------
trainer = Trainer(
    max_epochs=10,
    accelerator="mps" if torch.backends.mps.is_available() else "cpu",
    devices=1,
    gradient_clip_val=0.1,
)

# ----------------------------
# 🔁 8. 模型训练
# ----------------------------
trainer.fit(
    deepar,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)


You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name                   | Type                   | Params | Mode 
--------------------------------------------------------------------------
0 | loss                   | NormalDistributionLoss | 0      | train
1 | logging_metrics        | ModuleList             | 0      | train
2 | embeddings             | MultiEmbedding         | 12     | train
3 | rnn                    | LSTM                   | 14.1 K | train
4 | distribution_projector | Linear                 | 66     | train
--------------------------------------------------------------------------
14.2 K    Trainable params
0         Non-trainable params
14.2 K    Total params
0.057     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [8]:
from pytorch_forecasting.metrics import SMAPE, MAPE, RMSE

# 确保都在 CPU 上
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)]).cpu()
predictions = deepar.predict(val_dataloader).cpu()

# 指标也设置在 CPU
print("📊 Accuracy Metrics on Validation Set:")
print("  SMAPE:", SMAPE().to("cpu")(predictions, actuals).item())
print("  MAPE :", MAPE().to("cpu")(predictions, actuals).item())
print("  RMSE :", RMSE().to("cpu")(predictions, actuals).item())


You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


📊 Accuracy Metrics on Validation Set:
  SMAPE: 1.8410305976867676
  MAPE : 0.9572287797927856
  RMSE : 803.2193603515625
